In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('incident2.csv')
df.head()

,block_number,user_tx,fees
0,19412019,0x1ee8d8e23e4a026ce8afbf2dc79c196c6b1d43d5e9f8...,1.861322e+16
1,19412030,0x354e8386267ca643793de913739df3f9680895776f3a...,4.694753e+16
2,19412041,0xee8fd2c76181afa14ca0da158e0a01bba2d3df8e62c5...,6.182506e+16
3,19412043,0x5f3954cc3cb4fbb88803a910d852aab6566af2866acf...,1.283444e+17
4,19412043,0xbd9c86df3327871d630a0921c65cb7bec9b0c199d80c...,3.640861e+16


# use this api to find the position in block from transaction hash
# https://docs.infura.io/api/networks/ethereum/json-rpc-methods/eth_gettransactionreceipt


example: for the first transaction from the file we have, it has the transaction hash: 0x1ee8d8e23e4a026ce8afbf2dc79c196c6b1d43d5e9f8ebb80b968a367ed5e581

By looking at etherscan: 
- the user swapped  849 BRETT for 0.4 ETH on Uniswap v2
- from block 19412019

By looking at zeromev:
- the transaction was on position 2 (3d transaction)

In [3]:
# checking for that one transaction in infura

import requests
import json
from python_secrets import infura_api_key

url = f"https://mainnet.infura.io/v3/{infura_api_key}"

payload = json.dumps({
  "jsonrpc": "2.0",
  "method": "eth_getTransactionReceipt",
  "params": [
    "0x1ee8d8e23e4a026ce8afbf2dc79c196c6b1d43d5e9f8ebb80b968a367ed5e581"
  ],
  "id": 1
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"jsonrpc":"2.0","id":1,"result":{"blockHash":"0x2c1ca96b35f726a2171d3ebc050e390aca8dd1f83a2364ffa8cab9910143ee01","blockNumber":"0x1283433","contractAddress":null,"cumulativeGasUsed":"0x6f884","effectiveGasPrice":"0x11cd060113","from":"0x36a2ffb33c1b427c46c3d30adac3ca4e8ed36179","gasUsed":"0x2038c","logs":[{"address":"0xb0699d63aef20df3f1cffa9ca2bb8670416271d2","blockHash":"0x2c1ca96b35f726a2171d3ebc050e390aca8dd1f83a2364ffa8cab9910143ee01","blockNumber":"0x1283433","data":"0x00000000000000000000000000000000000000000000002e092d563911a14d34","logIndex":"0xd","removed":false,"topics":["0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef","0x00000000000000000000000036a2ffb33c1b427c46c3d30adac3ca4e8ed36179","0x000000000000000000000000e1ebdf64f7f3a31723e767a561345f958233bb7d"],"transactionHash":"0x1ee8d8e23e4a026ce8afbf2dc79c196c6b1d43d5e9f8ebb80b968a367ed5e581","transactionIndex":"0x2"},{"address":"0xb0699d63aef20df3f1cffa9ca2bb8670416271d2","blockHash":"0x2c1ca96b35f726a21

In [31]:
pd.DataFrame.from_dict(response.json()['result'], orient='index')

,0
blockHash,0x2c1ca96b35f726a2171d3ebc050e390aca8dd1f83a23...
blockNumber,0x1283433
contractAddress,None
cumulativeGasUsed,0x6f884
effectiveGasPrice,0x11cd060113
from,0x36a2ffb33c1b427c46c3d30adac3ca4e8ed36179
gasUsed,0x2038c
logs,[{'address': '0xb0699d63aef20df3f1cffa9ca2bb86...
logsBloom,0x00200000000000000000000080000000000000000000...
status,0x1


In [32]:
# we see that it was on position 0x2 hexa- decimal which is position 2 (third transaction)

# simulate the transaction if it was in the first block position

using phalcon blocksec we simulate if it was in position 0 in the block:
- block 19412019
- swap 807 BRETT for 0.4 ETH (instead of 849 if in position 3)

# calculate the difference (in the coin) between position we have vs. position 0

We have a difference of 42 BRETT

# Calculate the cost in usd and eth of the difference 
use dune
there is no BRETT token in dune - asked felix
do the analysis again with the 3 transaction (transaction hash 0xee8fd2c76181afa14ca0da158e0a01bba2d3df8e62c5da122ac1d77912aab66e) because there is the DMT token in dune